<a href="https://colab.research.google.com/github/amirabehery/AMIT/blob/main/AlexNet_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

# Importing Data & Splitting Data




In [2]:
# importing data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [3]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3)

In [4]:
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape, x_test.shape, y_test.shape

((35000, 32, 32, 3),
 (15000, 32, 32, 3),
 (35000, 1),
 (15000, 1),
 (10000, 32, 32, 3),
 (10000, 1))

# Image Augmentation

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True, zoom_range=0.1)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True, zoom_range=0.1)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True, zoom_range=0.1)

train_generator.fit(x_train)
val_generator.fit(x_valid)
test_generator.fit(x_test)

xtrain_generator = train_generator.flow(x_train, y_train)
xval_generator = val_generator.flow(x_valid, y_valid)
xtest_generator = test_generator.flow(x_test, y_test)

# Building Alex Model

In [6]:
alexNet = Sequential()

# 1st layer
alexNet.add(Conv2D(filters = 96, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same', activation='relu'))
alexNet.add(BatchNormalization())
alexNet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

# 2nd layer
alexNet.add(Conv2D(filters = 256, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu'))
alexNet.add(BatchNormalization())
alexNet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

# 3th layer
alexNet.add(Conv2D(filters = 384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
alexNet.add(BatchNormalization())

# 4th layer
alexNet.add(Conv2D(filters = 384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
alexNet.add(BatchNormalization())

# 5th layer
alexNet.add(Conv2D(filters = 256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
alexNet.add(BatchNormalization())
alexNet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

# flatten
alexNet.add(Flatten())

# Start Fully Connected Layer =====> 6th layer
alexNet.add(Dense(4096, activation='relu'))

# 1st dropout 
alexNet.add(Dropout(rate=0.5))

# 2nd Fully Connected Layer =====> 7th layer
alexNet.add(Dense(4096, activation='relu'))

# 2nd dropout 
alexNet.add(Dropout(rate=0.5))

# 3nd Fully Connected Layer =====> 8th layer
alexNet.add(Dense(4096, activation='relu'))

# output layer =====> 8th layer
alexNet.add(Dense(10, activation='softmax'))

In [7]:
alexNet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
alexNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8, 8, 96)          34944     
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 96)         384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 4, 96)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 256)         614656    
                                                                 
 batch_normalization_1 (Batc  (None, 4, 4, 256)        1024      
 hNormalization)                                                 
                                                        

In [8]:
alexNet.fit(xtrain_generator, epochs=10, validation_data=xval_generator, verbose=1)

Epoch 1/10
1094/1094 [==============================] - 51s 32ms/step - loss: 1.9860 - accuracy: 0.1028 - val_loss: 2.0781 - val_accuracy: 0.3627
Epoch 2/10
1094/1094 [==============================] - 48s 44ms/step - loss: 1.7630 - accuracy: 0.1060 - val_loss: 1.7785 - val_accuracy: 0.1288
Epoch 3/10
1094/1094 [==============================] - 35s 32ms/step - loss: 1.6194 - accuracy: 0.1009 - val_loss: 1.5603 - val_accuracy: 0.1270
Epoch 4/10
1094/1094 [==============================] - 52s 48ms/step - loss: 1.5176 - accuracy: 0.1113 - val_loss: 1.6602 - val_accuracy: 0.0935
Epoch 5/10
1094/1094 [==============================] - 57s 52ms/step - loss: 1.4468 - accuracy: 0.1104 - val_loss: 1.8316 - val_accuracy: 0.0770
Epoch 6/10
1094/1094 [==============================] - 40s 37ms/step - loss: 1.3640 - accuracy: 0.1073 - val_loss: 1.6863 - val_accuracy: 0.0697
Epoch 7/10
1094/1094 [==============================] - 40s 36ms/step - loss: 1.3117 - accuracy: 0.1056 - val_loss: 1.3587 -

In [14]:
y_pred = alexNet.predict(x_test)

313/313 [==============================] - 1s 4ms/step


In [15]:
def transform_predictions(list1):
  result = []
  for i in range(len(list1)):
    max_ = np.argmax(list1[i])
    result.append(max_)
  return result

In [16]:
y_pred_transformed = transform_predictions(y_pred)

In [19]:
accuracy_score(y_pred_transformed, y_test)

0.558